### Tutorial covering using Miniai for Image Classification

This tutorial covers how to use the miniai library for image classification.  It begins with Mnist and then moves to the Imagenet Tiny.  The evolution of the models includes the use of data augmentation techniques

### Import Libraies and setup

In [ ]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl,numpy as np,matplotlib.pyplot as plt
import fastcore.all as fc
from collections.abc import Mapping
from pathlib import Path
from operator import attrgetter,itemgetter
from functools import partial
from copy import copy
from contextlib import contextmanager

import torchvision.transforms.functional as TF,torch.nn.functional as F
from torch import tensor,nn,optim
from torch.utils.data import DataLoader,default_collate
from torch.nn import init
from torch.optim import lr_scheduler
from torcheval.metrics import MulticlassAccuracy
from datasets import load_dataset,load_dataset_builder

/home/miniai_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from miniai.datasets import DataLoaders, get_dls, collate_dict, inplace
from miniai.learner import Learner, TrainLearner, MetricsCB, DeviceCB, ProgressCB, LRFinderCB
from miniai.plotting import show_image, show_images
from miniai.model_blocks import conv, ResBlock, lin, pre_conv
from miniai.utils import set_seed
from miniai.activations import append_stats, ActivationStatsCB

### Set basic parameters and defaults

In [ ]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

import logging
logging.disable(logging.WARNING)

set_seed(42)

if fc.defaults.cpus>8: fc.defaults.cpus=8

### Load and configure Mnist Data 

In [ ]:
xl,yl = 'image','label'
name = "fashion_mnist"
bs = 1024
xmean,xstd = 0.28, 0.35

@inplace
def transformi(b): b[xl] = [(TF.to_tensor(o)-xmean)/xstd for o in b[xl]]

dsd = load_dataset(name)
tds = dsd.with_transform(transformi)
dls = DataLoaders.from_dd(tds, bs, num_workers=fc.defaults.cpus)

### Prepare Model callbacks, initialization, progress monitoring and metrics

In [ ]:
astats = ActivationStatsCB(append_stats, fc.risinstance(GeneralRelu))
metrics = MetricsCB(accuracy=MulticlassAccuracy())
cbs = [DeviceCB(), metrics, ProgressCB(plot=True), astats]
act_gr = partial(GeneralRelu, leak=0.1, sub=0.4)
iw = partial(init_weights, leaky=0.1)